In [1]:
# import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [12]:
for page_nr in range(1, pages_to_scrape+1):
    print(f"Scraping page nr. {page_nr:2}/{pages_to_scrape} (Posts found: {len(posts)}, total scraped: {len(posts_dict)})")

Scraping page nr  1/10 (Posts found: 20, total scraped: 212)
Scraping page nr  2/10 (Posts found: 20, total scraped: 212)
Scraping page nr  3/10 (Posts found: 20, total scraped: 212)
Scraping page nr  4/10 (Posts found: 20, total scraped: 212)
Scraping page nr  5/10 (Posts found: 20, total scraped: 212)
Scraping page nr  6/10 (Posts found: 20, total scraped: 212)
Scraping page nr  7/10 (Posts found: 20, total scraped: 212)
Scraping page nr  8/10 (Posts found: 20, total scraped: 212)
Scraping page nr  9/10 (Posts found: 20, total scraped: 212)
Scraping page nr 10/10 (Posts found: 20, total scraped: 212)


In [ ]:
a = {"1234_2": {"text": "abc", "count": 2}, 
     "7567": {"text": "klm", "count": 2},}
b = {"1234_2": {"author": "def", "type": "xyz"}}

print(a.items())

{key: values["text"] for key, values in a.items()}

dict_items([('1234_2', {'text': 'abc', 'count': 2}), ('7567', {'text': 'klm', 'count': 2})])


ValueError: too many values to unpack (expected 2)

In [93]:
pages_to_scrape = 2

BASE_URL = "https://www.boredpanda.com/page/"

main_page_urls = [f"{BASE_URL}{p}/" for p in range(1, pages_to_scrape+1)]
main_page_urls

['https://www.boredpanda.com/page/1/', 'https://www.boredpanda.com/page/2/']

In [ ]:
# Set-Up
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Cafari/537.36'}
pages_to_scrape = 2
posts_dict = {}

# Iterate through pages
for page_nr in range(1, pages_to_scrape+1):
    url = f"https://www.boredpanda.com/page/{page_nr}/"
    r = requests.get(url, headers=headers)
    
    soup = BeautifulSoup(r.text, 'html.parser')
    posts = soup.find_all("article", class_="post")

    print(f"Scraping page nr. {page_nr:2}/{pages_to_scrape} (Posts found: {len(posts)}, total scraped: {len(posts_dict)})")

    # Iterate through each post on the current page loaded.
    for post in posts:
        # Get post id. If already existent in dict, add suffix.
        post_id = post["data-post-id"]
        while post_id in posts_dict:
            post_id += "_2"
            
        # Access post to retrieve categories, date and number of postings.
        post_url = post.find("a")["href"]
        post_r = requests.get(post_url, headers=headers)
        
        post_soup = BeautifulSoup(post_r.text, 'lxml')

        post_categories_date = post_soup.find("div", class_="categories-list")
        post_categories = post_categories_date.find("div", class_="categories").text.strip()
        post_date = post_categories_date.find("div", class_="post-dates").text.strip()
        posts_total = post_soup.find_all("div", class_="open-list-header")
        posts_shown = post_soup.find_all("div", class_="open-list-item open-list-block clearfix")
        nr_postings_total = len(posts_total)
        nr_postings_shown = len(posts_shown)

        # Add post info to data dictionairy
        posts_dict.update(
            {post_id: {
                "title": post.find("a", class_="title").text.strip(),
                #"categories": post["data-categories"],
                "voting": int(post.find("p", class_="votes-text").text.strip()),
                "nr_comments": (int(post.find("p", class_="comment-text").text.strip())
                    if post.find("p", class_="comment-text") else None),
                "nr_postings_total": nr_postings_total,
                "nr_postings_shown": nr_postings_shown,
                "date_posted": post_date,
                "categories": post_categories,
                "has_experimental_title": post["data-content-experiment-enabled"],
                "page_nr": page_nr,
                "link": post_url
            }}
        )

Scraping page nr.  1/2 (Posts found: 33, total scraped: 0)
Scraping page nr.  2/2 (Posts found: 20, total scraped: 33)


In [ ]:
#50.2s

In [17]:
results = pd.DataFrame(posts_dict).T
results.index.name = "post_id"
results.head(10)

,title,categories,voting,nr_comments,nr_postings,date_posted,post_categories,has_experimental_title,link
post_id,,,,,,,,,
8046358,Woman’s Viral ‘Medical Binder’ Idea Has People...,"feminism,social-issues",34,9,0,"Jan 25, 2025","Feminism, Social Issues",true,https://www.boredpanda.com/doctor-medical-bind...
8053723,RIP Brain Rot: Guy’s Viral Theory Suggests A M...,work-money,28,3,0,"Jan 25, 2025",Work & Money,true,https://www.boredpanda.com/post-brain-rot-era-...
8044900,Wife Gives Husband An Ultimatum Over Stepdaugh...,"family,relationships",30,8,0,"Jan 25, 2025","Family, Relationships",true,https://www.boredpanda.com/father-refuses-wife...
8084910,Justin Baldoni Shoots Down Blake Lively’s Gag ...,"celebrities-news,news",-15,5,0,"Jan 24, 2025","Celebrities, News",false,https://www.boredpanda.com/justin-baldoni-resp...
8084152,“Get One Yourself!”: Mom Unplugged Baby’s NICU...,"news,world",3,31,0,"Jan 24, 2025","News, World",false,https://www.boredpanda.com/mom-unplugs-babys-n...
8081543,Billionaire Dad Shares Son’s Nighttime Stats i...,"lifestyle,news",-7,16,0,"Jan 24, 2025","Lifestyle, News",false,https://www.boredpanda.com/billionaire-trying-...
8084800,Contents Of VIP Goodie Bags At Trump Inaugurat...,news,-7,10,0,"Jan 24, 2025",News,false,https://www.boredpanda.com/trump-inauguration-...
8080021,Audrey Hepburn’s Son Breaks Silence On Ivanka ...,"news,us",-23,28,0,"Jan 24, 2025","News, US",false,https://www.boredpanda.com/audrey-hepburn-son-...
8080536,“Pretty Damning”: Hollywood Intimacy Coordinat...,"celebrities-news,news",-29,16,0,"Jan 24, 2025","Celebrities, News",false,https://www.boredpanda.com/intimacy-coordinato...


In [18]:
results.to_excel("results_check_inkl_post_data.xlsx", header=True, index=True)

# TESTS

In [29]:
post_url = "https://www.boredpanda.com/awkward-parents-kids-conversations/"
post_r = requests.get(post_url, headers=headers)
# post_soup = BeautifulSoup(post_r.text, 'html.parser') # 2.4s
# post_soup = BeautifulSoup(post_r.text, 'lxml') # 1.0s
post_soup = BeautifulSoup(post_r.text, 'xml') # AttributeError: 'NoneType' object has no attribute 'find'
print(post_soup)

# category_and_date = post_soup.find("div", class_="categories-list")

# print("category:", category_and_date.find("div", class_="categories").text.strip())
# print("date:", category_and_date.find("div", class_="post-dates").text.strip())

# posts = post_soup.find_all("div", class_="open-list-item open-list-block clearfix")
# post_headers = post_soup.find_all("div", class_="open-list-header")
# print(len(posts))
# print(len(post_headers))

# category: Funny, Jokes
# date: Jan 22, 2025
# 49
# 108

<?xml version="1.0" encoding="utf-8"?>



In [124]:

post_urls = ["https://www.boredpanda.com/unfortunate-public-interior-designs/",
             "https://www.boredpanda.com/awkward-parents-kids-conversations/",
             "https://www.boredpanda.com/dogs-in-street-photography-streetphotographersfdn/",
             "https://www.boredpanda.com/awkward-parents-kids-conversations/"]


#    <p class="author-role"> <span class="member-type community">Author, </span> <span>BoredPanda staff</span> </p>


for post_url in post_urls:
    post_r = requests.get(post_url, headers=headers)
    post_soup = BeautifulSoup(post_r.text, 'html.parser')

    category_and_date = post_soup.find("div", class_="categories-list")

    print("category:", category_and_date.find("div", class_="categories").text.strip())
    print("date:", category_and_date.find("div", class_="post-dates").text.strip())

    posts = post_soup.find_all("div", class_="open-list-item open-list-block clearfix")
    post_headers = post_soup.find_all("div", class_="open-list-header")
    print("posts_shown:", len(posts))
    print("posts_total:", len(post_headers))

    #authors = post_soup.find("div", class_="post-authors-content")
    authors = post_soup.find_all("a", class_="author")
    print(len(authors))
    authors_str = " | ".join([a.text.strip() for a in authors])
    authors_type = post_soup.find("span", class_="member-type community")
    authors_type2_ls = post_soup.find_all("span", class_="member-type community")
    authors_type2 = ", ".join([at2.text for at2 in authors_type2_ls])
    authors_role_ls = post_soup.find_all("p", class_="author-role")
    authors_role = ", ".join([ar.text for ar in authors_role_ls])
    authors_role2 = [ar.text for ar in authors_role_ls]

    print("authors_str:", authors_str)
    print("authors_type:", authors_type.text.strip())
    print("authors_type2:", authors_type2)
    print("authors_role:", authors_role)
    print("authors_role2:", authors_role2)
    print("\n")

category: Fails, Funny
date: Jan 21, 2025
posts_shown: 39
posts_total: 100
3
authors_str: Eglė Bliabaitė, Kornelija Viečaitė, Jonas Zvilius
authors_type: Author,
authors_type2: Author, , Author, , Writer, , Writer, , Author, , Author, 
authors_role:  Author,  BoredPanda staff ,  Author,  BoredPanda staff ,  Writer,  BoredPanda staff ,  Writer,  BoredPanda staff ,  Author,  BoredPanda staff ,  Author,  BoredPanda staff 
authors_role2: [' Author,  BoredPanda staff ', ' Author,  BoredPanda staff ', ' Writer,  BoredPanda staff ', ' Writer,  BoredPanda staff ', ' Author,  BoredPanda staff ', ' Author,  BoredPanda staff ']


category: Funny, Jokes
date: Jan 22, 2025
posts_shown: 49
posts_total: 108
3
authors_str: Aurelija Rakauskaitė, Austeja Zokaite, Rugile Baltrunaite
authors_type: Author,
authors_type2: Author, , Author, , Writer, , Writer, , Author, , Author, 
authors_role:  Author,  BoredPanda staff ,  Author,  BoredPanda staff ,  Writer,  BoredPanda staff ,  Writer,  BoredPanda staff ,

In [90]:
post_url = "https://www.boredpanda.com/dogs-in-street-photography-streetphotographersfdn/"
post_r = requests.get(post_url, headers=headers)
post_soup = BeautifulSoup(post_r.text, 'html.parser')

category_and_date = post_soup.find("div", class_="categories-list")

print("category:", category_and_date.find("div", class_="categories").text.strip())
print("date:", category_and_date.find("div", class_="post-dates").text.strip())

posts = post_soup.find_all("div", class_="open-list-item open-list-block clearfix")
post_headers = post_soup.find_all("div", class_="open-list-header")

authors = post_soup.find_all("a", class_="author")
print(len(authors))
authors_str = ", ".join([a.text for a in authors])
authors_type = post_soup.find("span", class_="member-type community")

print(authors_str)
print(authors_type.text.strip())
print(len(posts))
print(len(post_headers))

category: Animals, Dogs
date: Jan 24, 2025
1
Hidrėlėy
Pro member
35
35


In [91]:
post_url = "https://www.boredpanda.com/awkward-parents-kids-conversations/"
post_r = requests.get(post_url, headers=headers)
post_soup = BeautifulSoup(post_r.text, 'html.parser')

category_and_date = post_soup.find("div", class_="categories-list")

print("category:", category_and_date.find("div", class_="categories").text.strip())
print("date:", category_and_date.find("div", class_="post-dates").text.strip())

posts = post_soup.find_all("div", class_="open-list-item open-list-block clearfix")
post_headers = post_soup.find_all("div", class_="open-list-header")

authors = post_soup.find_all("a", class_="author")
print(len(authors))
authors_str = ", ".join([a.text for a in authors])
authors_type = post_soup.find("span", class_="member-type community")

print(authors_str)
print(authors_type.text.strip())
print(len(posts))
print(len(post_headers))

category: Funny, Jokes
date: Jan 22, 2025
3
Aurelija Rakauskaitė, Austeja Zokaite, Rugile Baltrunaite
Author,
49
108


## ASYNC PARSING

In [42]:
urls = results["link"].to_list()[:2]
len(urls)

2

In [65]:
import aiohttp
import asyncio
import concurrent.futures
from bs4 import BeautifulSoup

async def fetch(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            return await response.text()

async def parse(html_doc, loop):
    post_soup = await loop.run_in_executor(None, BeautifulSoup, html_doc, 'html.parser')

    post_categories_date = post_soup.find("div", class_="categories-list")
    post_categories = post_categories_date.find("div", class_="categories").text.strip()
    post_date = post_categories_date.find("div", class_="post-dates").text.strip()
    posts_total = post_soup.find_all("div", class_="open-list-header")
    posts_shown = post_soup.find_all("div", class_="open-list-item open-list-block clearfix")
    nr_postings_total = len(posts_total)
    nr_postings_shown = len(posts_shown)
    print(post_categories, post_date, nr_postings_total, nr_postings_shown)
    return [post_categories, post_date, nr_postings_total, nr_postings_shown]


async def main(url):
    html = await fetch(url)
    output = await parse(html, loop)
    print(output)
    # tasks = []
    # for url in urls:
    #     result = await loop.run_in_executor(None, parse(fetch(url)))
    #     return await result
    #     task = asyncio.create_task(parse(fetch(url)))
    #     tasks.append(task)
    # await asyncio.gather(*tasks)


url=urls[0]

await main(url)

# loop = asyncio.get_event_loop()
# loop_result = loop.run_until_complete(main(loop))
# loop.close()
# loop_result = loop.run_until_complete(main())
#asyncio.run(main(url))

Feminism, Social Issues Jan 25, 2025 0 0
['Feminism, Social Issues', 'Jan 25, 2025', 0, 0]


c:\Users\nickm\AppData\Local\Programs\Python\Python311\Lib\site-packages\bs4\element.py:955: RuntimeWarning: coroutine 'main' was never awaited
  u.setup()


In [114]:
import time

time.time()

1737920831.1241124

In [117]:
import time

start = time.strftime('%X')
start = time.time()
time.sleep(3)
end = time.strftime('%X')
end = time.time()

print(start, end, end-start)

1737920891.0076556 1737920894.0129297 3.0052740573883057


In [130]:
df = pd.read_excel("c:/Users/nickm/Desktop/Coding/Projects/blog-post-sentiment-analysis/scraping_asyncio_test2.xlsx", index_col=0)

df["page_link"].value_counts()

page_link
https://www.boredpanda.com/page/1/    32
https://www.boredpanda.com/page/4/    20
https://www.boredpanda.com/page/3/    20
https://www.boredpanda.com/page/2/    20
https://www.boredpanda.com/page/5/    20
Name: count, dtype: int64

In [135]:
a = 2

print(f"banana{'s' if a > 1 else ''}")

bananas
